## Optimization

This notebook introduces our work regarding optimizing the SGHMC sampling using `Numba` and `Cython`.

In [1]:
import numpy as np
import numpy.linalg as la
import cython
import timeit
import warnings
warnings.filterwarnings('ignore')

### Plain Python Implementation

In [2]:
def sghmc(theta0, gradU, V, alpha = 0.1, epoch = 500, step = 1, lr = 0.01):
    '''Simpler sghmc sampling'''
    
    np.random.seed(1234)
    
    p = theta0.shape[0]
    beta = 0.5 * lr * V
    disp = la.cholesky(2 * lr * (alpha * np.eye(p) - 2 * beta))
    
    samples = np.zeros((epoch, p))
    samples[0] = theta0
    
    for i in range(epoch - 1):
        theta = samples[i]
        r = np.random.randn(p)
        
        for j in range(step):
            
            theta += lr * r
            r += -lr * gradU(theta) - alpha * r + disp @ np.random.randn(p)
        
        samples[i + 1] = theta
    
    return samples

In [3]:
# set the parameters

U = lambda x: -2 * x**2 + x**4
gradU =  lambda x: -4 * x + 4 * x**3 + 2 * np.random.randn()
theta0 = np.zeros((2))
V = 4 * np.eye((2))

### Profiling part

In [4]:
%prun -q -D sghmc.prof sghmc(theta0, gradU, V)

 
*** Profile stats marshalled to file 'sghmc.prof'. 


In [5]:
import pstats

p = pstats.Stats('sghmc.prof')
p.print_stats()
pass

Sun Apr 25 10:07:38 2021    sghmc.prof

         2025 function calls in 0.037 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.037    0.037 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
      499    0.008    0.000    0.009    0.000 <ipython-input-3-4658113d012f>:4(<lambda>)
        1    0.000    0.000    0.037    0.037 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'seed' of 'numpy.random.mtrand.RandomState' objects}
     1497    0.008    0.000    0.008    0.000 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
        1    0.000    0.000    0.000    0.000 {method '__array_prepare__' of 'numpy.ndarray' objects}
        1   

From the profiling result we can see that `np.random.randn` and `lambda` function are called the most times during running the sghmc function. This could help us to find which parts to focus on and then optimize. Before using `numba` and `cython` to optimize, we need to store the sampling result and its cost time.

In [6]:
samples = sghmc(theta0, gradU, V)

In [7]:
%timeit sghmc(theta0, gradU, V)

13.7 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Numba

We use `numba.jit` as a function decorator to speed up the `sghmc` function 

In [8]:
import numba
from numba import jit

In [9]:
@jit
def gradU_numba(x):
    '''Gradient function with numba.jit'''
    
    return -4 * x + 4 * x**3 + 2 * np.random.randn()

@jit
def sghmc_numba(theta0, gradU, V, alpha = 0.1, epoch = 500, step = 1, lr = 0.01):
    '''sghmc sampling with numba'''
    
    np.random.seed(1234)
    
    p = theta0.shape[0]
    beta = 0.5 * lr * V
    disp = la.cholesky(2 * lr * (alpha * np.eye(p) - 2 * beta))
    
    samples = np.zeros((epoch, p))
    samples[0] = theta0
    
    for i in range(epoch - 1):
        theta = samples[i]
        r = np.random.randn(p)
        
        for j in range(step):
            
            theta += lr * r
            r += -lr * gradU(theta) - alpha * r + disp @ np.random.randn(p)
        
        samples[i + 1] = theta
    
    return samples

In [10]:
print(np.allclose(samples, sghmc_numba(theta0, gradU_numba, V)))

True


In [11]:
%timeit sghmc_numba(theta0, gradU_numba, V)

633 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We can see that when we change the lambda function to a defined function with a `JIT` decorator, the speed here is much faster than in the plain implementation. This also proves that the bottemneck of the whole function is from the gradient calculation (It is hard to optimize `numpy.random.randn`). Next we will use `cython` to do the optimization.

In [12]:
%load_ext cython

In [25]:
%%cython -a

import cython
import scipy.linalg as la
import numpy as np
cimport numpy as np
from libc.math cimport sqrt, pow

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef gradU_cython(double[:] x):
    '''Gradient function with cython'''
    
    cdef int n
    cdef double[:] grad
    
    n = x.shape[0]
    grad = np.zeros(n, dtype = 'double')
    
    for i in range(n):
        grad[i] = -4 * x[i] + 4 * pow(x[i], 3) + 2 * np.random.randn()
    
    return grad

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def sghmc_cython(double[:] theta0, double[:, :] V, double alpha = 0.1, \
                 int epoch = 500, int step = 1, double lr = 0.01):
    
    '''sghmc sampling with cython'''
    
    cdef int i, j, k, a, b
    cdef int p = theta0.shape[0]
    cdef double[:, :] disp = np.zeros((p, p), dtype = 'double')
    cdef double[:, :] samples = np.zeros((epoch, p), dtype = 'double')
    cdef double[:] theta = np.zeros(p, dtype = 'double')
    cdef double[:] r = np.zeros(p, dtype = 'double')
    cdef double[:] grad = np.zeros(p, dtype = 'double')
    cdef double[:] randres = np.zeros(p, dtype = 'double')
    cdef double[:] dotres = np.zeros(p, dtype = 'double')
    
    np.random.seed(1234)
    for i in range(p):
        for j in range(p):
            if i == j:
                disp[i, j] = alpha - lr * V[i, j]
            else:
                disp[i, j] = -lr * V[i, j]
            disp[i, j] *= 2 * lr
    
    disp = la.cholesky(disp)
    
    for i in range(p):
        samples[0, i] = theta0[i]
        
    for i in range(epoch - 1):
        theta = samples[i, :].copy()
        r = np.random.randn(p)
        
        for j in range(step):
            for k in range(p):
                theta[k] += lr * r[k]
            
            grad = gradU_cython(theta)
            randres = np.random.randn(p)
            for a in range(p):
                for b in range(p):
                    dotres[a] += disp[a, b] * randres[b]
            
            for k in range(p):
                r[k] += -lr * grad[k] - alpha * r[k] + dotres[k]
        
        for j in range(p):
            samples[i + 1, j] = theta[j]
    
    return samples          

In [24]:
%timeit sghmc_cython(theta0, V)

6.26 ms ± 414 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can see that the speed here is around twice as fast as the plain python implementation, but it is still much slower than the `JIT` version. One of the possible reasons is that our implementation is still not so fast as the one optimized by `JIT`. The bottleneck is still from the gradient calculation, and in practice we can call some gradient function from `tensorflow`, which could make the function faster.